In [2]:
# imports
#!pip install tables
#!pip install fastparquet
from fastparquet import write

from tables import *
import openai  # for generating embeddings
import pandas as pd  # for DataFrames to store article sections and embeddings
import re  # for cutting <ref> links out of Wikipedia articles
import tiktoken  # for counting tokens
import numpy as np
from dotenv import dotenv_values

In [236]:
#To use embeddings you must create an .env file where the content is OPENAI_API_KEY = "your-api-key"
config = dotenv_values(".env")["OPENAI_API_KEY"]
openai.organization = "org-3O7bHGD9SwjHVDuUCNCGACC3"
openai.api_key = config

In [237]:
df = pd.read_csv("legal_acts_estonia.csv", names=['type', 'nr','text','link'])

In [238]:
df

,type,nr,text,link
0,VVS,para1,§ 1.\nVabariigi Valitsuse pädevus\n(1) Vabarii...,https://www.riigiteataja.ee/akt/VVS#para1
1,VVS,para2,§ 2.\nVabariigi Valitsuse asukoht\nVabariigi V...,https://www.riigiteataja.ee/akt/VVS#para2
2,VVS,para3,§ 3.\nVabariigi Valitsuse liikmed\n(1) Vabarii...,https://www.riigiteataja.ee/akt/VVS#para3
3,VVS,para3b1,§ 3.1.\nVabariigi Valitsuse liikme juurdepääs ...,https://www.riigiteataja.ee/akt/VVS#para3b1
4,VVS,para4,§ 4.\nVabariigi Valitsuse liikmete tööülesanne...,https://www.riigiteataja.ee/akt/VVS#para4
...,...,...,...,...
32784,NaN,para7,§ 7.\nKaitsekulu\nKaitsekulu on NATO meetodi j...,https://www.riigiteataja.ee/akt/#para7
32785,NaN,para8,§ 8.\nEesti Haigekassa eelarvepositsioon\nEest...,https://www.riigiteataja.ee/akt/#para8
32786,NaN,para9,§ 9.\nEelarveaasta jooksul riigiteede üleandmi...,https://www.riigiteataja.ee/akt/#para9
32787,NaN,para10,§ 10.\nTapa spordi- ja vabaajakeskuse ehituse ...,https://www.riigiteataja.ee/akt/#para10


In [239]:
df["text"][0]

'§\xa01.\nVabariigi Valitsuse pädevus\n(1) Vabariigi Valitsus teostab täidesaatvat riigivõimu Eesti Vabariigi põhiseaduse ja seaduste alusel.\n(2) Vabariigi Valitsus teostab täidesaatvat riigivõimu vahetult või valitsusasutuste kaudu.\n'

In [240]:
df["title"] = df["text"].str.split("\n").str[1]
df["text"] = df["text"].str.split("\n").apply(lambda x: ','.join(x[2:]))

In [241]:
df["text"] = df["text"].str.replace('\n','')

In [242]:
#have to split up to have less than 1600 words
df['split_text'] = df['text'].str.split(r'\(\d+\)')


In [243]:
df["split_text"][0]

['',
 ' Vabariigi Valitsus teostab täidesaatvat riigivõimu Eesti Vabariigi põhiseaduse ja seaduste alusel.,',
 ' Vabariigi Valitsus teostab täidesaatvat riigivõimu vahetult või valitsusasutuste kaudu.,']

In [244]:
df = df.explode("split_text")

In [245]:
df = df[df["split_text"]!= ""]

In [246]:
def count_words(text):
    return len(text.split())
df['word_count'] = df['split_text'].apply(count_words)

In [247]:
df_cut = df[["type","nr","link","title","split_text"]]
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use
EMBEDDING_MODEL = "text-embedding-ada-002"

In [249]:
df["concatenated"] = df["title"]+ " "+ df["split_text"]
df["concatenated"] = df["concatenated"].str.replace(r'\s+', ' ').str.rstrip(",")


C:\Users\annilo\AppData\Local\Temp\ipykernel_16368\1343037904.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["concatenated"] = df["concatenated"].str.replace(r'\s+', ' ').str.rstrip(",")


In [253]:
df_less= df[df["word_count"]<500]

In [259]:
laws = np.array(df_less["concatenated"])
#laws = laws[:10000]

In [ ]:
# calculate embeddings

##
##DO NOT RUN UNLESS NEED TO CREATE NEW EMBEDDINGS (THIS CODE COSTS ABT 2 DOLLARS)

#EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI's best embeddings as of Apr 2023
#BATCH_SIZE = 1000  # you can submit up to 2048 embedding inputs per request
#
#wikipedia_strings = laws.tolist()
#
#embeddings = []
#for batch_start in range(0, len(wikipedia_strings), BATCH_SIZE):
#    batch_end = batch_start + BATCH_SIZE
#    batch = wikipedia_strings[batch_start:batch_end]
#    print(f"Batch {batch_start} to {batch_end-1}")
#    response = openai.Embedding.create(model=EMBEDDING_MODEL, input=batch)
#    for i, be in enumerate(response["data"]):
#        assert i == be["index"]  # double check embeddings are in same order as input
#    batch_embeddings = [e["embedding"] for e in response["data"]]
#    embeddings.extend(batch_embeddings)
#
#df = pd.DataFrame({"text": wikipedia_strings, "embedding": embeddings})

In [ ]:
#df['text'] = df['text'].astype('str')
#df['embedding'] = df['embedding'].astype('str')

In [8]:
#otherwise loading is super slow
#df.to_hdf(r'embeddedfile_2.h5', key='stage', mode='w')

In [3]:
#reread = pd.read_hdf('./embeddedfile.h5')  

In [6]:
#reread = reread[:1000]

In [7]:
#reread.to_csv("sample_embeddings.csv")